# Final Project (30%)

For the final project, you will need to implement a "new" statistical algorithm in Python from the research literature and write a "paper" describing the algorithm. 

Deadline 2nd May 2018 at 11:59 PM

Note: 1 bonus point for each day that it is submitted before the deadline. The actual project has a maximum grade of 100, but bonus points can push it above 100.

## Paper

The paper should have the following:

### Title

Should be consise and informative.

### Abstract

250 words or less. Identify 4-6 key phrases.

### Background

State the research paper you are using. Describe the concept of the algorithm and why it is interesting and/or useful. If appropriate, describe the mathematical basis of the algorithm. Some potential topics for the backgorund include:

- What problem does it address? 
- What are known and possible applications of the algorithm? 
- What are its advantages and disadvantages relative to other algorithms?
- How will you use it in your research?

### Description of algorithm

First, explain in plain English what the algorithm does. Then describes the details of the algorihtm, using mathematical equations or pseudocode as appropriate. 

### Describe optimization for performance

First implement the algorithm using plain Python in a straightforward way from the description of the algorihtm. Then profile and optimize it using one or more apporpiate mathods, such as:

1. Use of better algorithms or data structures
2. Use of vectorization
3. JIT or AOT compilation of critical functions
4. Re-writing critical functions in C++ and using pybind11 to wrap them
5. Making use of parallelism or concurrency
6. Making use of distributed compuitng

Document the improvemnt in performance with the optimizations performed.

### Applications to simulated data sets

Are there specific inputs that give known outuputs (e.g. there might be closed form solutions for special input cases)? How does the algorithm perform on these? 

If no such input cases are available (or in addition to such input cases), how does the algorithm perform on simulated data sets for which you know the "truth"? 

### Applications to real data sets

Test the algorithm on the real-world examples in the orignal paper if possible. Try to find at least one other real-world data set not in the original paper and test it on that. Describe and interpret the results.

### Comparative analysis with competing algorihtms

Find two other algorihtms that addresss a similar problem. Perform a comparison - for example, of accurary or speed. You can use native libraires of the other algorithms - you do not need to code them yourself. Comment on your observations. 

### Discussion/conclusion

Your thoughts on the algorithm. Does it fulfill a particular need? How could it be generalized to other problem domains? What are its limiations and how could it be improved further?

### References/bibliography

Make sure you cite your sources.

## Code

The code should be in a public GitHub repository with:

1. A README file
2. An open source license
3. Source code
4. Test code
5. Examples
6. A reproducible report

The package should be downloadable and installable with `python setup.py install`, or even posted to PyPI adn installable with `pip install package`.


## Rubric

Each item is worth 10 points, but some sections will give up to 10 bonus points if done really well. Note that the "difficulty factor" of the chosen algorithm will be factored into the grading. 

1. Is the abstract, background and discussion readable and clear? (10-20 points)
2. Is the algorithm description clear and accurate? (10-20 points)
3. Has the algorihtm been optimized? (10-20 points)
4. Are the applicaitons to simulated/real data clear and useful? (10-20 points)
5. Was the comarative analysis done well? (10-20 points points)
6. Is there a well-maitnatined Github repository for the code? (10 points)
7. Is the document show evidenc of literate programming? (10 points)
8. Is the analyiss reproducible? (10 points)
9. Is the code tested? Are examples provided? (10 points)
10. Is the package easily installable? (10 points)



In [5]:
import numpy as np
import scipy.stats
import scipy.linalg

### Hamiltonian Monte Carlo

In [2]:
# Suppose U is given
def U(x):
    return -2*x**2 + x**4

def grad_U(x):
    return -4*x + 4*x**3

def H(theta, r ,U, M_inv):
    '''Hamiltonian function of total energy'''
    return U(theta) + 0.5 * r.T @ M_inv @ r

In [26]:
# Suppose U is computed from given prior and xmodel

# test: a normal prior, suppose to be given
def prior(theta):
    return scipy.stats.norm.pdf(theta)

def xmodel(x, theta):
    return scipy.stats.norm.pdf(x, loc = theta)

def U(X, theta, xmodel, prior):
    '''compute U from prior, xmodel, and data X'''
    # X should be a vetor
    temp = 0
    for x in X:
        temp += np.log(xmodel(x, theta))
    return - temp - np.log(prior(theta))

def grad_U(X, theta, xmodel, prior, h = 0.0001):
    '''compute the gradient of U'''
    d = theta.shape[0]
    ans = []
    u0 = U(X, theta, xmodel, prior)
    for i in range(d):
        theta0 = theta
        theta0[i] += h
        u1 = U(X, theta0, xmodel, prior)
        print((u0, u1))
        ans.append( (u1-u0)/h )
    return ans
        

In [36]:
X = np.array([[1.],[2.]])
theta = np.array([1.])
grad_U(X, theta, xmodel, prior, h = 0.00000001) # this is sensitive to choice of h, check!

(array([ 3.7568156]), array([ 3.7568156]))


[array([ -4.44089210e-08])]

In [57]:
def hmc(theta_0, eps, niter, m, M, U, grad_U, prior, xmodel):
    """Hamiltonian Monte Carlo"""
    d = theta_0.shape[0]
    Theta = np.empty((d, niter))
    R = np.empty((d, niter))
    M_inv = scipy.linalg.inv(M)
    theta = theta_0
    for i in range(niter):
        r = np.random.multivariate_normal(np.zeros(d), M, 1).flatten()      
        theta_old = theta
        r_old = r
            
        #discretize Hamiltonian dynamics
        r -= eps / 2 * grad_U(theta)
        for j in range(m):
            theta += eps * M_inv @ r
            r -= eps * grad_U(theta)  
        r -= eps / 2 * grad_U(theta)
        
        #M-H correction
        u = np.random.uniform(0, 1, 1)
        rho = np.exp(H(theta, r ,U, M_inv)-H(theta_old, r_old ,U, M_inv))        
        if u < rho:
            Theta[:,i] = theta
            R[:,i] = r
        else:
            Theta[:,i] = theta_old
            R[:,i] = r_old
        theta = Theta[:,i]
        
    return Theta, R

In [58]:
#This should be in the main function

#user inputs:
theta_0 = np.array([1.])
eps = 0.01
niter = 100
M = np.array([1.]).reshape(1,1)
m = 100

Theta, R = hmc(theta_0, eps, niter, m, M, U, grad_U)

### Stochastic gradient HMC

In [ ]:
def sghmc(theta_0, eps, niter, m, M, U, grad_U，perc, prior, xmodel):
    """Stochastic Gradient Hamiltonian Monte Carlo"""
    # theta_0: the starting point
    # eps: step length in solving the dynamic system
    # niter: the number sampling
    # m: the maximum number of steps to take for solving the Hamiltonian
    # M: the mass matrix in the Hamiltonian
    # perc: the percentage of minbatch size over total size
    d = theta_0.shape[0]
    batch_size = int(np.floor(X.shape[0] * perc))
    Theta = np.empty((d, niter))
    R = np.empty((d, niter))
    M_inv_eps = scipy.linalg.inv(M) * eps
    theta = theta_0
    M0 = C @ M_inv_eps
    V = 2*eps*(C - B_hat)
    for i in range(niter):
        r = np.random.multivariate_normal(np.zeros(d), M, 1).flatten()      
        for j in range(m):
            theta += M_inv_eps @ r           
            X0 = X[np.random.randint(batch_size, size = (batch_size,)),] #sample X
            r = r - eps * grad_U(X0, theta, xmodel, prior, h = 0.0001) - M0 @ r + np.random.multivariate_normal(np.zeros(d), V, 1)        
        Theta[:,i] = theta
        R[:,i] = r
        
    return Theta, R